# WorkSpaceの作成

In [10]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

# これが無いとエラーが出たので追加
# tenant_idはActive Directory の プロパティより確認
interactive_auth = InteractiveLoginAuthentication(tenant_id="bcd8db96-8bb9-4f0d-af35-e471bf92c072")

from azureml.core import Workspace
ws = Workspace.create(name='Azureml-SDK-WS01',
                     subscription_id='3467f739-a57b-4612-9de8-72a6616c01b3',
                     resource_group='AzuremlSDKRG00',
                     create_resource_group=True,
                     location='japaneast',
                     auth=interactive_auth)

Deploying KeyVault with name azuremlskeyvaultca514745.
Deploying AppInsights with name azuremlsinsights7d67e3c7.
Deployed AppInsights with name azuremlsinsights7d67e3c7. Took 6.24 seconds.
Deploying StorageAccount with name azuremlsstorage74437b7d7.
Deployed KeyVault with name azuremlskeyvaultca514745. Took 22.43 seconds.
Deploying Workspace with name Azureml-SDK-WS01.
Deployed StorageAccount with name azuremlsstorage74437b7d7. Took 30.94 seconds.
Deployed Workspace with name Azureml-SDK-WS01. Took 33.68 seconds.


In [12]:
# Azure PortalからConfigファイルをダウンロードする
# もしくは以下のコードを実行
ws.write_config(path="./config")

# データストアの作成

In [13]:
from azureml.core import Workspace, Datastore

# ワークスペースにアクセスする
ws = Workspace.from_config(path='./config')

# データストアの作成
az_store = Datastore.register_azure_blob_container(
    workspace=ws,
    datastore_name='azure_sdk_blob01',
    account_name='dp100udemy', #storage accountの名前
    container_name='data',
    account_key='32cMFMxXzFskFrYFo84e1etBWDDZvUecgxgsXR7w+14M73Ih5lZqVN2VCoTAWp3oDqtMWI6QQCog+AStU5YYbQ==')

# データセットの作成

In [18]:
# データセットの作成
from azureml.core import Workspace, Datastore, Dataset

# ワークスペースにアクセスする
ws = Workspace.from_config(path='./config')

# データストアにアクセス
az_store = Datastore.get(ws, "azure_sdk_blob01")

In [19]:
# csvへのpathを作成
# 複数ある場合は,で複数入力可能
csv_path = [(az_store, "data/Loan Approval Prediction.csv")]

# データセットの作成
loan_dataset = Dataset.Tabular.from_delimited_files(path=csv_path)

# データセットの登録
loan_dataset = loan_dataset.register(workspace=ws,
                                    name='Loan Applications Using SDK',
                                    create_new_version=True) # もし存在しているならnew_versionとして作成する

# オブジェクトへのアクセス

In [21]:
# workspaceのリストを取得する
ws_list = Workspace.list(subscription_id='3467f739-a57b-4612-9de8-72a6616c01b3')

In [23]:
ws_list = list(ws_list)
ws_list

['Azureml-SDK-WS01', 'dp100udemy']

In [28]:
# デフォルトのデータストアにアクセスする
az_default_store = ws.get_default_datastore()
az_default_store

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-27bbdeda-49d1-4f8b-9a75-71d17bc330d8",
  "account_name": "azuremlsstorage74437b7d7",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [29]:
# データストアのリストを取得する
store_list = list(ws.datastores)
store_list

['azure_sdk_blob01',
 'workspacefilestore',
 'workspaceblobstore',
 'workspaceworkingdirectory',
 'workspaceartifactstore']

In [31]:
# データのリストを取得する
ds_list = list(ws.datasets.keys())
ds_list

['Loan Applications Using SDK']

In [30]:
# データセットを取得する
az_dataset = Dataset.get_by_name(ws, "Loan Applications Using SDK")
az_dataset

{
  "source": [
    "('azure_sdk_blob01', 'data/Loan Approval Prediction.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "44437c86-0df7-44b8-b1f2-e567d7cb91bf",
    "name": "Loan Applications Using SDK",
    "version": 1,
    "workspace": "Workspace.create(name='Azureml-SDK-WS01', subscription_id='3467f739-a57b-4612-9de8-72a6616c01b3', resource_group='AzuremlSDKRG00')"
  }
}

# DataFrame

In [37]:
ws = Workspace.from_config('./config')
az_store = Datastore.get(ws, 'azure_sdk_blob01')
az_dataset = Dataset.get_by_name(ws, 'Loan Applications Using SDK')

In [38]:
# pandas データフレーム
df = az_dataset.to_pandas_dataframe()

In [40]:
# データセットにアップロードする
df_sub = df[['Married', 'Gender', 'Loan_Status']]
az_ds_from_df = Dataset.Tabular.register_pandas_dataframe(dataframe=df_sub,
                                                         target=az_store,
                                                         name='Loan Dataset From Dataframe')

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/28783968-8e3c-4da4-8dc4-0e410cf83157/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


# ローカルファイルのアップロード

In [44]:
files_list = ['./data/test.csv', './data/test1.csv']
az_store.upload_files(files=files_list,
                     target_path='Data/',
                     relative_root='./data/', # 複数ファイルがこの中に入る
                     overwrite=True)

Uploading an estimated of 2 files
Uploading ./data/test.csv
Uploaded ./data/test.csv, 1 files out of an estimated total of 2
Uploading ./data/test1.csv
Uploaded ./data/test1.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_021589ba99514c849f1a4317448cdb7d

In [45]:
# フォルダーのアップロード
az_store.upload(src_dir='./data',
               target_path='Data/data',
               overwrite=True)

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 2 files
Uploading ./data\test.csv
Uploaded ./data\test.csv, 1 files out of an estimated total of 2
Uploading ./data\test1.csv
Uploaded ./data\test1.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_2480040aa53e433f80477e156f64efd6